In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os 

In [2]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=12,                     # Number of cores per task (adjust if needed)
    processes=6,                 # One process per task
    memory="100GB",               # --mem
    walltime="01:15:00",         # --time
    queue="med",               # --partition
    log_directory=".",           # Logs will be saved to the current directory
)



/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43149 instead
  warnings.warn(


In [3]:
from dask.distributed import Client


cluster.scale(jobs=10)
client = Client(cluster)


In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.42.239.61:43149/status,
Dashboard: http://10.42.239.61:43149/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.42.239.61:37413,Workers: 0
Dashboard: http://10.42.239.61:43149/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [21]:
infol = '/home1/nalex2023/Datasets/era5_manus/'

files = glob.glob(infol+'*.nc')

dft_era = pd.DataFrame(files, columns=['file'])


dft_era['datetime'] = dft_era['file'].str.split(os.sep).str[-1].str.split('_').str[2].str.split('m').str[0]


dft_era['datetime'] = pd.to_datetime(dft_era['datetime'], format='%Y%m')

dft_era['month'] = dft_era['datetime'].dt.month

# seasonal DJF , MAM, JJA, SON
seasonal = dft_era[dft_era['month'].isin([9,10,11])]

seasonal


,file,datetime,month
0,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2003-10-01,10
5,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2011-11-01,11
7,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2010-11-01,11
9,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2002-10-01,10
14,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2012-11-01,11
17,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2001-10-01,10
21,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2004-10-01,10
24,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2005-10-01,10
30,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2007-10-01,10
32,/home1/nalex2023/Datasets/era5_manus/era5_plev...,2006-10-01,10


In [22]:
def prepro(ds):
    ds_sub = ds[['u','v']].sel(pressure_level=slice(1000,800),latitude=slice(0,-5),longitude=slice(141,154))
    return ds_sub


dset_seasonal = xr.open_mfdataset(seasonal['file'], combine='by_coords',chunks='auto', preprocess=prepro,
                                  parallel=True)




In [23]:
dset_seasonal

<xarray.Dataset> Size: 557MB
Dimensions:         (valid_time: 26928, pressure_level: 3, latitude: 21,
                     longitude: 41)
Coordinates:
    number          int64 8B 0
  * valid_time      (valid_time) datetime64[ns] 215kB 2001-09-01 ... 2013-09-...
  * pressure_level  (pressure_level) float64 24B 1e+03 900.0 800.0
  * latitude        (latitude) float64 168B 0.0 -0.25 -0.5 ... -4.5 -4.75 -5.0
  * longitude       (longitude) float64 328B 141.0 141.2 141.5 ... 150.8 151.0
    expver          (valid_time) <U4 431kB dask.array<chunksize=(720,), meta=np.ndarray>
Data variables:
    u               (valid_time, pressure_level, latitude, longitude) float32 278MB dask.array<chunksize=(480, 2, 2, 41), meta=np.ndarray>
    v               (valid_time, pressure_level, latitude, longitude) float32 278MB dask.array<chunksize=(480, 2, 2, 41), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-24T02:59 GRIB to CDM+CF via cfgrib-0.9.1...

In [24]:
dset_seasonal_diurnal = dset_seasonal.groupby('valid_time.hour').mean()


In [25]:
out_dir = '/home1/nalex2023/Datasets/era5_manus/climt/'
dset_seasonal_diurnal.to_netcdf(out_dir+'era5_diurnal_SON.nc')


In [10]:
dset_seasonal_diurnal.close()